**load package**

In [1]:
import pandas as pd
import json
import spacy
import re
nlp = spacy.load('en_core_web_sm')

**1. Format the data**

In [ ]:
data = []
path = "./Cholesterol_submissions.json" # change this to your submission json file path
for line in open(path, 'rb'):
    data.append(json.loads(line))

In [ ]:
df=pd.DataFrame.from_dict(data)
df["created_utc"]=df["created_utc"].astype(int)
df["score"]=df["score"].astype(int)
df["num_comments"]=df["num_comments"].astype(int)

In [ ]:
df = df[["author","author_created_utc","created_utc","id","num_comments","score","upvote_ratio","title","selftext","subreddit"]][(df["author"]!="[deleted]")]

In [ ]:
df.info()

In [ ]:
df = df.drop(df[(df["author"] =="Therinicus")|(df["author"] =="draka1")| (df["author"] =="_alexkane_")|(df["author"] =="AutoModerator")|(df["author"] =="Bojarow")].index)
df = df.drop(df[(df["selftext"] =="[removed]")| (pd.isnull(df["selftext"]))].index)
df = df.reset_index(drop=True)

In [ ]:
df.head()

**2. Data Preprocessing**

extract submissions with identified statin names

In [ ]:
def check_sentence(sentence, group):
    # Convert the sentence to lowercase for case-insensitive matching
    sentence = sentence.lower()

    # Tokenize the sentence and apply spaCy's linguistic annotations
    doc = nlp(sentence)

    # Lemmatize each word in the sentence and convert plural to singular
    words = [token.lemma_ for token in doc]

    # Check if there is at least one word from group 1 in the modified sentence
    has_group_word = any(word in words for word in group)

    return has_group_word 

In [ ]:
med = ["statin","atorvastatin","simvastatin","rosuvastatin","pitavastatin","fluvastatin","lovastatin","pravastatin","lipitor","zocor","crestor","livalo","lescol","mevacor","pravachol"]

In [ ]:
df.insert(9, 'text', True)

In [ ]:
df["title"]=df["title"].astype("str")
df["selftext"]=df["selftext"].astype("str")

In [ ]:
df["text"]=df["title"].str.cat(df["selftext"],sep=" ")

In [ ]:
df.insert(10,"statins", False)

In [ ]:
for i in range(len(df)):
    sentence=df.loc[i,["text"]][0]
    if check_sentence(sentence, med):
        df.loc[i,["statins"]] = True

In [ ]:
df_statins= df[df["statins"] == True]
df_statins.reset_index(drop=True, inplace=True)
df_statins.head()

In [ ]:
df_statins.insert(10,"extracted_text", True)

extract nouns, verbs and adjectives from texts

In [ ]:
for i in range(len(df_statins)):
    sentence=df_statins.loc[i,["text"]][0]
    doc = nlp(sentence)
    filtered_tokens = [token for token in doc if token.pos_ in ["VERB", "NOUN", "ADJ", "PROPN"] and len(token.text) > 1 and not token.is_digit]
    new_sentence = " ".join([token.text.lower() for token in filtered_tokens])
    df_statins.loc[i,["extracted_text"]]=new_sentence

In [ ]:
df_statins.head()

In [ ]:
df_statins.to_csv("./Cholesterol_statins.csv") #change this to your saving file path